> **Live Demo**
> ---

Here we will extract the generated optimal parameters from the saved file to reduce time needed to present the results. We will present 20 images results each.


# Imports, Data and Functions

Importing essential packages, loading image data and defining necessary functions

In [ ]:
!pip install scikit-optimize

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.8/107.8 kB 2.6 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.io as sio
from tqdm import tqdm

from PIL import Image, ImageEnhance
from skimage.util import img_as_float
from skimage.color import rgb2gray
from skimage.io import imread, imsave
from skimage.measure import shannon_entropy
from skimage.feature import local_binary_pattern
import cv2

from sklearn.metrics import f1_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV

from skopt import gp_minimize
from skopt.space import Categorical

from google.colab import drive

In [ ]:
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
# data_dir = # "PATH_TO_YOUR_FOLDER"

# Checking if our specified directory exists
os.path.exists(data_dir)

True

In [ ]:
class BoundaryDataset(object):
    """
    Project boundary dataset wrapper

    Given the path to the root of the dataset, this class provides
    methods for loading images and ground truths.

    Attributes:

    root_dir - the root path of the dataset
    data_path - the path of the data directory within the root
    sample_names - a list of names of images
    """
    def __init__(self, root_dir='.', split='train'):
        """
        Constructor

        :param root_dir: the path to the root of the custom dataset
        :param split: 'train' or 'test'
        """
        self.root_dir = root_dir
        self.data_path = os.path.join(root_dir, split)
        self.sample_names = self._sample_names(self.data_path)

    def __len__(self):
        """
        Get the number of samples in the dataset
        :return: the number of samples
        """
        return len(self.sample_names)

    @staticmethod
    def _sample_names(directory):
        names = []
        files = os.listdir(directory)
        for fn in files:
            name, ext = os.path.splitext(fn)
            if ext.lower() == '.jpg':
                names.append(name)
        return names

    def read_image(self, name):
        """
        Load the image identified by the sample name
        :param name: the sample name
        :return: a (H,W,3) array containing the image
        """
        path = os.path.join(self.data_path, f"{name}.jpg")
        return imread(path)

    def load_boundaries(self, name):
        """
        Load the boundaries identified by the sample name
        :param name: the sample name
        :return: a list of (H,W) arrays, each of which contains a boundary ground truth
        """
        boundary_path = os.path.join(self.data_path, f"{name}.npy")
        if os.path.exists(boundary_path):
            boundaries = np.load(boundary_path, allow_pickle=True)
            return list(boundaries)
        return []

In [ ]:
split = 'train'

# Load the dataset using BoundaryDataset class
train_dataset = BoundaryDataset(data_dir + "/project_data", split=split)

print(f"Data size: {len(train_dataset)}")

Data size: 200


In [ ]:
split = 'test'

# Load the dataset using BoundaryDataset class
test_dataset = BoundaryDataset(data_dir + "/project_data", split=split)

print(f"Data size: {len(test_dataset)}")

Data size: 100


In [ ]:
# Function to generate edges for each channel (RGB) and combine them
def generate_edges_rgb(image, blur_amounts, sigmas, contrast_factors):
    # Separate the channels
    red_channel = image[:, :, 2]  # Red channel
    green_channel = image[:, :, 1]  # Green channel
    blue_channel = image[:, :, 0]  # Blue channel

    # Process each channel independently
    def process_channel(channel, blur_amount, sigma, contrast_factor):

        blurred = cv2.GaussianBlur(channel, (blur_amount, blur_amount), sigma)

        img_pil = Image.fromarray(blurred)
        enhanced_img_pil = ImageEnhance.Contrast(img_pil).enhance(contrast_factor)
        enhanced_img = np.array(enhanced_img_pil)

        grad_x = cv2.Sobel(enhanced_img, cv2.CV_64F, 1, 0, ksize=3)
        grad_y = cv2.Sobel(enhanced_img, cv2.CV_64F, 0, 1, ksize=3)
        gradient_magnitude = np.sqrt(grad_x**2 + grad_y**2)

        # Normalize the gradient magnitude to [0, 1]
        gradient_magnitude_min = np.min(gradient_magnitude)
        gradient_magnitude_max = np.max(gradient_magnitude)
        normalized_gradient = (gradient_magnitude - gradient_magnitude_min) / (gradient_magnitude_max - gradient_magnitude_min)

        return normalized_gradient

    red_edges = process_channel(red_channel, blur_amounts[0], sigmas[0], contrast_factors[0])
    green_edges = process_channel(green_channel, blur_amounts[1], sigmas[1], contrast_factors[1])
    blue_edges = process_channel(blue_channel, blur_amounts[2], sigmas[2], contrast_factors[2])

    # Combine the edges from all channels
    combined_edges = np.maximum.reduce([red_edges, green_edges, blue_edges])

    return combined_edges

# Presenting Results

## Train

Presenting Image, Boundaries and Predicted Edges for train images

In [ ]:
optimal_parameters = np.load(os.path.join(data_dir, 'saved_variables', 'optimal_parameters_train.npy'), allow_pickle=True).item()
selected_sample_names = train_dataset.sample_names[:20]

for selected_sample_name in selected_sample_names:
    print(f"Processing image: {split}/{selected_sample_name}")
    image = train_dataset.read_image(selected_sample_name)
    boundaries = train_dataset.load_boundaries(selected_sample_name)

    # retrieve optimal parameters that were predicted
    blur_amounts, sigmas, contrast_factors = optimal_parameters[selected_sample_name]

    # generate edges based on optimal parameters
    edges = generate_edges_rgb(image, blur_amounts, sigmas, contrast_factors)

    # display image
    fig, ax = plt.subplots(1, 5, figsize=(10, 8))
    ax[0].imshow(image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    # display boundaries (ground truth)
    for i in range(len(boundaries)):
        ax[i + 1].imshow(boundaries[i], cmap='gray')
        ax[i + 1].set_title(f"Boundary {i + 1}")
        ax[i + 1].axis("off")
        edge_count = np.sum(boundaries[i] > 0)
        edge_density = edge_count / (boundaries[i].size)

    # display extracted edges
    ax[4].imshow(edges, cmap='gray')
    ax[4].set_title("Extracted Edge Strength")
    ax[4].axis("off")

    plt.tight_layout()
    plt.show()

Output hidden; open in https://colab.research.google.com to view.

## Test

Presenting Image and Predicted Edges for train images

In [ ]:
optimal_parameters = np.load(os.path.join(data_dir, 'saved_variables', 'optimal_parameters.npy'), allow_pickle=True).item()
selected_sample_names = test_dataset.sample_names[:20]

for selected_sample_name in selected_sample_names:
    print(f"Processing image: {split}/{selected_sample_name}")
    image = test_dataset.read_image(selected_sample_name)

    # retrieve optimal parameters
    blur_amounts, sigmas, contrast_factors = optimal_parameters[selected_sample_name]

    # generate edges based on optimal parameters
    edges = generate_edges_rgb(image, blur_amounts, sigmas, contrast_factors)

    # display images and boundaries and extracted edges
    fig, ax = plt.subplots(1, 2, figsize=(5, 3))
    ax[0].imshow(image)
    ax[0].set_title("Original Image")
    ax[0].axis("off")

    ax[1].imshow(edges, cmap='gray')
    ax[1].set_title("Extracted Edge Strength")
    ax[1].axis("off")

    plt.tight_layout()
    plt.show()

Output hidden; open in https://colab.research.google.com to view.